# Imports

In [ ]:
import glob
import itertools as it
import os
import re
from collections import namedtuple
from functools import partial
from pathlib import Path

import dask
import distributed
import h5py
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from holoviews.operation.datashader import regrid
from tqdm.auto import tqdm, trange

IDX = pd.IndexSlice

In [ ]:
from dask.diagnostics import ProgressBar

ProgressBar().register()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.image_analysis.mosaic as mosaic
from paulssonlab.image_analysis import *
from paulssonlab.image_analysis.ui import display_image

In [ ]:
%load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/230213/230213induction.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230215/230215induction.nd2" #v=7
# filename = "/home/jqs1/scratch/jqs1/microscopy/230326/230326promrbs.nd2" #v=8,t=10
# filename = "/home/jqs1/scratch/jqs1/microscopy/230404/230404_rbsprom.nd2"
# filename = workflow.SplitFilename(
#     sorted(
#         glob.glob(
#             "/home/jqs1/scratch/jqs1/microscopy/230619/230619_NAO745_repressilators_split.nd2*"
#         )
#     )
# )
filename = workflow.SplitFilename(
    sorted(
        glob.glob(
            "/home/jqs1/scratch/jqs1/microscopy/230707/230707_repressilators_restart.nd2.split.a*"
        )
    )
)
fish_filename = Path(filename).parent / "FISH/real_run"

In [ ]:
output_dir = Path(filename).parent / "test_output"
output_dir.mkdir(exist_ok=True)

In [ ]:
dataset = ds.dataset(
    output_dir / "test2/measurements", format="parquet", partitioning="hive"
)

In [ ]:
%%time
# df = dataset.to_table(filter=ds.field("position") == 11).to_pandas()
df = dataset.to_table().to_pandas()

In [ ]:
df.info(verbose=True, memory_usage="deep")

In [ ]:
%%time
df2 = (
    df[df["observable"] == "p90"]
    .pivot_table(
        columns=["channel"], values=["value"], index=["position", "roi_idx", "t"]
    )
    .droplevel(0, axis=1)
    # .droplevel(["position"])
    # .reset_index("roi_idx")
).loc[:, ["CFP-EM", "YFP-EM", "RFP-EM"]]

In [ ]:
df2.info(verbose=True, memory_usage="deep")

In [ ]:
df2.head()

In [ ]:
df2.to_parquet(output_dir / "test2/measurements_gaurav.parquet", index=True)

In [ ]:
(output_dir / "test2/measurements_gaurav.parquet")

In [ ]:
!du -hs /home/jqs1/scratch/jqs1/microscopy/230707/test_output/test2/measurements_gaurav.parquet

In [ ]:
list(df2.groupby(["position", "roi_idx"]))[1]

In [ ]:
x = norm(df2.loc[(11, 21)].loc[IDX[40:], :])

In [ ]:
pd.plotting.autocorrelation_plot(x["CFP-EM"])

In [ ]:
df2.reset_index()[df2.reset_index()["roi_idx"] == 244]

In [ ]:
norm(df2.loc[(11, 244)].loc[IDX[40:], :]).hvplot()

In [ ]:
(df2.index.get_level_values("roi_idx") == 244).sum()

In [ ]:
roi_idxs = np.unique(
    df2.index.get_level_values("roi_idx")
)  # df2.index.levels[df2.index.names.index("roi_idx")]

In [ ]:
hv.HoloMap(
    {
        t: norm(
            groups[t][1].droplevel(["position", "roi_idx"]).loc[IDX[40:140], :]
        ).hvplot(color=channel_colors)
        for t in range(2)
    }
)

In [ ]:
osc_roi_idxs = [
    21,
    57,
    67,
    103,
    105,
    107,
    116,
    149,
    162,
    170,
    185,
    191,
    215,
    237,
    246,
    252,
    268,
    285,
    302,
    319,
    321,
    342,
    346,
    375,
    417,
    432,
    453,
    454,
    457,
    462,
    463,
    535,
    567,
    588,
    600,
    638,
    644,
    650,
    677,
    680,
    690,
    707,
]

control_roi_idxs = [
    404,
    415,
    420,
    421,
    429,
    434,
    440,
    443,
    444,
    486,
    500,
    530,
    580,
    585,
    593,
    611,
    617,
    620,
    624,
    627,
    629,
    632,
    634,
    642,
    651,
    677,
    682,
    687,
    696,
    731,
    733,
]

In [ ]:
(len(osc_roi_idxs), len(control_roi_idxs))

In [ ]:
import imageio.v3 as iio

In [ ]:
!rm -rf $output_dir/materials

In [ ]:
(output_dir / "materials").mkdir()

In [ ]:
hvplot.extension("matplotlib")

In [ ]:
# prefix = "control_"
prefix = ""
for roi_idx in tqdm(osc_roi_idxs):
    # for roi_idx in tqdm(control_roi_idxs):
    x = z[roi_idx, :t_max, :]
    x = trim_axes(x, x != 0)
    kymograph = np.concatenate(
        [
            image.colorize(
                image.unstack(norm(x[40:180, i], 0.999))[np.newaxis],
                [channel_colors[i]],
            )
            for i in range(ary.shape[1])
        ],
        axis=0,
    )
    iio.imwrite(
        output_dir / f"materials/{prefix}kymograph_{roi_idx}.png",
        (np.clip(kymograph, 0, 1) * 255).astype(np.uint8),
        extension=".png",
    )
    trace_plot = (
        norm(df2.loc[(11, roi_idx)].loc[IDX[40:180], :])
        .hvplot(color=channel_colors)
        .redim(value="fluorescence")
    )
    hv.save(
        trace_plot,
        output_dir / f"materials/{prefix}plot_{roi_idx}.png",
        fmt="png",
        dpi=200,
        backend="matplotlib",
    )

In [ ]:
groups = df2.groupby(["position", "roi_idx"], as_index=False, group_keys=False)

In [ ]:
hv.HoloMap(
    {
        idx[1]: norm(group.droplevel(["position", "roi_idx"]).loc[IDX[40:], :]).hvplot(
            color=channel_colors
        )
        for idx, group in groups
    }
)

In [ ]:
(output_dir / "materials").mkdir()

In [ ]:
hvplot.extension("matplotlib")

In [ ]:
plot = (
    norm(df2.loc[(11, 67)].loc[IDX[40:180], :])
    .hvplot(color=channel_colors)
    .redim(value="fluorescence")
)
hv.save(
    plot, output_dir / "materials/test1.png", fmt="png", dpi=200, backend="matplotlib"
)